## Tide Prediction Module Functions

 - retrieve_constituents - retrieves harmonic constituents from NOAA gauge station
 - <code>retrieve_water_levels()</code> - retrieves observed water levels from NOAA's API
 - <code>retrieve_predicted_tide()</code> - retrieves predicted tide from NOAA's API
 - datetimes - prepares a collection of datetimes from beginning to end dates
 
 - <code>predict_tide()</code> - predicts tide for desired NOAA station given station ID, start date and end date for prediction 
 - datum_value - retrieves datum value for desired datum reference, hardcoded for <b>MTL</b>
 - <code>detide()</code> - detides observed water levels
 - <code>surge()</code> - predicts surge at NOAA gauge station provided station ID, start date, end date, and landfall date, best for Clawpack Simulation!

# Example of Tide Prediction For One Date Instance

- In this example, method used to predict tide is adapated from Pytides
- This implementation will only work for known NOAA gauge stations
- Harmonic Constituents data is scraped from NOAA. 

In [ ]:
import matplotlib.pyplot as plt
import datetime
import tide 

### ****  Station Information ****

Locate NOAA station ID.  NOAA gauge stations home: https://tidesandcurrents.noaa.gov/ <br>
Fill in station ID and date for tide prediction!

In [ ]:
#Station Information
stationID = '8518750'

#Date of prediction (YEAR, MTH, DAY, HR)
prediction_date = datetime.datetime(2017,10,5,0)

### Tide Prediction

Prediction of tide at specified location (station ID) and specified time (GMT) implemented below by calling <b>predict_tide( )</b> method with the following arguments: <b> stationID, beg_prediction_date, end_prediction_date</b>. <br>

To predict tide at an instant, set <b>beg_prediction_date</b> and <b>end_prediction_date</b> in <b>predict_tide( )</b> method to the same date!

In [ ]:
#NOAA Data Scraping Implementation      
height = tide.predict_tide(stationID, prediction_date, prediction_date) # in meters
print(height[0], "meters")

*******************************************************************************************************************

# Example of Tide Prediction In A Date Interval 

###   Station Information 

Fill in station ID, a beginning date and an end date for tide prediction below

In [ ]:
#Station Information
stationID = '8518750'

#Beginning and End Dates 
beg_date = datetime.datetime(2017,10,1,0,0)
end_date = datetime.datetime(2017,10,5,0,0)

#Predict tide with arguments set as: (stationID, beg_prediction_date, end_prediction_date)
predicted_tide = tide.predict_tide(stationID, beg_date, end_date)

### Tide Predictions
Plot results in a time series plot

In [ ]:
#Method datetimes() makes a range of datetimes given arguments: (beg_prediction_date, end_prediction_date)
times = tide.datetimes(beg_date, end_date)

plt.figure(figsize=(20,10))
plt.plot(times, predicted_tide, "-", label="Tide Prediction")
plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
plt.ylabel('Meters'), plt.margins(x=0), plt.legend(loc = 'best')
plt.title('My Prediction for Station {}'.format(stationID))
plt.show()

*******************************************************************************************************************

# Example Comparing NOAA vs Our Tide Prediction In A Date Interval 

In [ ]:
#Station Information
stationID = '8518750'

#Beginning and End Dates 
beg_date = datetime.datetime(2017,10,1,0,0)
end_date = datetime.datetime(2017,10,2,0,0)

#Predict Tide 
predicted_tide = tide.predict_tide(stationID, beg_date, end_date)

- Calling function <b>retrieve_water_levels( )</b> with arguments set as: <b>(stationID, beg_prediction_date, end_prediction_date)</b> retrieves and downloads NOAA's datetimes and observed water level data for the specified NOAA station in the date interval provided
- The function <b>retrieve_predicted_tide( )</b> arguments set as: <b>(stationID, beg_prediction_date, end_prediction_date)</b> retrieves and downloads NOAA's predicted tide data for the specified NOAA station. 
- Data is scraped in <b>Metric</b> units, <b>GMT </b> timezone, <b>MSL</b> datum and  <b>6 min</b> interval. 

In [ ]:
times, NOAA_observed_water_lvl = tide.retrieve_water_levels(stationID, beg_date, end_date)
NOAA_predicted_tide = tide.retrieve_predicted_tide(stationID, beg_date, end_date)

In [ ]:
#Plot Comparisons
plt.figure(figsize=(20,10))
plt.plot(times, predicted_tide, "-", label="Our Tide Prediction")
plt.plot(times, NOAA_predicted_tide, "-", label="NOAA Tide Prediction")
plt.plot(times, NOAA_observed_water_lvl, "-", label="NOAA Water Level Observation")
plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
plt.ylabel('Metres'), plt.margins(x=0), plt.legend(loc = 'best')
plt.title('Comparison of Our Prediction vs NOAA prediction for Station {}'.format(stationID))
plt.show()

*******************************************************************************************************************

# Example Detiding and Capturing A Surge for a Gauge Station 

- Calling <b>predict_tide( )</b> method with arguments set as: <b>(stationID, beg_prediction_date, end_prediction_date)</b> will output predicted tide at the specified location and time interval
- Calling <b>retrieve_water_levels( )</b> method with arguments set as: <b>(stationID, beg_prediction_date, end_prediction_date)</b> with output datetimes and observed water levels at the specified location and time interval
- Calling <b>detide( )</b> method with arguments set as: <b>(NOAA observed water level, predicted tide)</b> will output detided water level. 

In [ ]:
#Station Information
stationID = '8518750'

#Beginning and End Dates 
beg_date = datetime.datetime(2017,10,1,0,0)
end_date = datetime.datetime(2017,10,5,0,0)

predicted_tide = tide.predict_tide(stationID, beg_date, end_date)
times, NOAA_observed_water_lvl = tide.retrieve_water_levels(stationID, beg_date, end_date)

surge = tide.detide(NOAA_observed_water_lvl, predicted_tide)

#Plot Comparisons
plt.figure(figsize=(20,10))
plt.plot(times, surge, "-", label="Our Surge Prediction")
plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
plt.ylabel('Metres'), plt.margins(x=0), plt.legend(loc = 'best')
plt.title('Detided Water Level for Station {}'.format(stationID))
plt.show()

*******************************************************************************************************************

# Example for Clawpack Storm Surge Implementation

- Uncomment line below to utilize Clawpack's pytides module located under geoclaw
- Code below works best if placed in <b>gauge_afteraxes( )</b> in <b>setplot.py</b>
- Calling <b>surge( )</b> method with arguments set as: <b>(stationID, beginning_date, end_date, landfall_date)</b> will output observed surge from NOAA.

In [ ]:
import clawpack.geoclaw.tide as tide
import datetime

In [ ]:
#Station Information
stationID = '8518750'

#Beginning, End, Landfall Dates 
beg_date = datetime.datetime(2017,10,1,0,0)
end_date = datetime.datetime(2017,10,5,0,0)
landfall_date = datetime.datetime(2017,10,3,12,0)

# Surge Prediction
times, surge = tide.surge(stationID, beg_date, end_date, landfall_date)
plt.plot(times, surge, color="b", label="Our Prediction")

*******************************************************************************************************************

# Example Iterating Through A Library Of Stations And Date Intervals

In [ ]:
import clawpack.geoclaw.tide as tide
from datetime import datetime

In [ ]:

station_dict = {'8518750':(datetime(2017,10,1,0), datetime(2017,10,5,0), datetime(2017,10,3,0)),
                '8540433':(datetime(2017,9,1,0), datetime(2017,9,10,12), datetime(2017,9,5,6)),
                '8531680':(datetime(2020,10,1,0), datetime(2020,10,10,0), datetime(2020,10,6,0)),
                '8722956':(datetime(2019,8,1,0), datetime(2019,8,12,0), datetime(2019,8,6,12)),
                '8658120':(datetime(2018,11,1,0), datetime(2018,11,8,0), datetime(2018,11,3,18)),
                '8516945':(datetime(2013,1,1,0), datetime(2013,1,6,0), datetime(2013,1,3,12))}

for (key, value) in station_dict.items():
    stationID = key
    beg_date = value[0]
    end_date = value[1]
    landfall_date = value[2]
    
    #NOAA Data Scraping Implementation
    predicted_tide = tide.predict_tide(stationID, beg_date, end_date)         
    times, NOAA_observed_water_lvl = tide.retrieve_water_levels(stationID, beg_date, end_date)
    NOAA_predicted_tide = tide.retrieve_predicted_tide(stationID, beg_date, end_date)
    
    #Detide Water Level
    surge = tide.detide(NOAA_observed_water_lvl, predicted_tide)
    NOAA_surge = tide.detide(NOAA_observed_water_lvl, NOAA_predicted_tide)
    
    #Plot Comparisons
    plt.figure(figsize=(20,10))
    plt.plot(times, predicted_tide, "-", label="Our Tide Prediction")
    plt.plot(times, NOAA_predicted_tide, "-", label="NOAA Tide Prediction")
    plt.plot(times, NOAA_observed_water_lvl, "-", label="NOAA Water Level Observation")
    plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
    plt.ylabel('Metres'), plt.margins(x=0), plt.legend(loc = 'best')
    plt.title('Comparison of Our Prediction vs NOAA prediction for Station {}'.format(stationID))
    plt.show()
    
    #Detided Water Level Comparison
    plt.figure(figsize=(20,10))
    plt.plot(times, surge, "-", label="Our Detided Prediction")
    plt.plot(times, NOAA_surge, "-", label="NOAA's Detided Prediction")
    plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
    plt.ylabel('Metres'), plt.margins(x=0), plt.legend(loc = 'best')
    plt.title('Detided Water Level Comparison of Our Prediction vs NOAA prediction for Station {}'.format(stationID))
    plt.show()
    
    
    #### Clawpack Implementation (in setplot.py) ####
    times, surge = tide.surge(stationID, beg_date, end_date, landfall_date)
    plt.plot(times, surge, color="b", label="Our Surge Prediction")
    
    